<a href="https://colab.research.google.com/github/zulesu/probable-giggle/blob/main/copy_of_gcp_auth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Authenticate to GCP

Colab allows you to perform operations on Google Cloud Platform via GCP APIs. You must first authenticate to a project you have the appropriate permissions to. Learn more about GCP projects [here](https://cloud.google.com/resource-manager/docs/creating-managing-projects).

In [ ]:
from google.colab import auth
PROJECT_ID = "gen-lang-client-0361273389" # @param {type: "string"}
auth.authenticate_user(project_id=PROJECT_ID)

In [ ]:
!pip uninstall -y datasets gcsfs bigframes
!pip install fsspec==2024.9.0
!pip install datasets==3.1.0 gcsfs==2024.9.0 bigframes==1.27.0
!apt-get install -y python3-cairo


Found existing installation: gcsfs 2024.10.0
Uninstalling gcsfs-2024.10.0:
  Successfully uninstalled gcsfs-2024.10.0
Found existing installation: bigframes 1.29.0
Uninstalling bigframes-1.29.0:
  Successfully uninstalled bigframes-1.29.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
INFO: pip is looking at multiple versions of fsspec[http] to determine which version is compatible with other requirements. This could take a while.
Reason for being yanked: requirements incorrect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.1/755.1 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:

In [ ]:
import logging
import torch
import wandb
import os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, IntervalStrategy

# Postavljanje W&B API ključa
os.environ['WANDB_API_KEY'] = '0dd449c9c7a50feb94ef49dbbede869392953b8f'

# Ručni login na W&B ako je potrebno
wandb.login()

# Pokrenite W&B sesiju sa tačnim imenom entiteta i projektom
wandb.init(project="openAI-gpt2", entity="rudicstipan5-lipilipic", settings=wandb.Settings(init_timeout=120))

# Logger setup
logger = logging.getLogger(__name__)
logger.info("Starting script execution...")

# Environment varijabla za izbegavanje fragmentacije memorije CUDA
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Kreiranje potrebnih direktorijuma
output_dir = '/content/drive/My Drive/gpt2'
checkpoint_dir = os.path.join(output_dir, 'results')

for directory in [output_dir, checkpoint_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)
        logger.info(f"Created directory: {directory}")


# Funkcija za pronalaženje poslednjeg validnog checkpoint-a
def get_last_valid_checkpoint(directory):
    logger.info(f"Searching for the last valid checkpoint in the output directory {directory}.")
    if not os.path.exists(directory):
        logger.error(f"Output directory {directory} does not exist.")
        return None
    checkpoints = [os.path.join(directory, d) for d in os.listdir(directory) if d.startswith('checkpoint-')]
    if not checkpoints:
        logger.info("No checkpoints found.")
        return None
    last_checkpoint = max(checkpoints, key=os.path.getctime)
    if os.path.isdir(last_checkpoint):
        logger.info(f"Last valid checkpoint: {last_checkpoint}")
        return last_checkpoint
    else:
        logger.info("Valid checkpoint directory not found.")
        return None

# Učitaj dataset
dataset = load_dataset('Triangle104/Guilherme34-uncensor')
logger.info("Dataset loaded successfully.")
print(dataset)

# Učitaj tokenizer i model
model_name = 'gpt2-medium'  # Korišćenje većeg modela
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
logger.info("Tokenizer and model loaded successfully.")
tokenizer.pad_token = tokenizer.eos_token


# Funkcija za tokenizaciju
def tokenize_function(examples):
    messages = examples['messages']
    if isinstance(messages, list) and isinstance(messages[0], list) and isinstance(messages[0][0], dict):
        messages = [" ".join([msg['content'] for msg in conversation]) for conversation in messages]
    elif isinstance(messages, list) and isinstance(messages[0], dict):
        messages = " ".join([msg['content'] for msg in messages])
    elif not isinstance(messages, list):
        messages = [messages]
    tokenized_inputs = tokenizer(messages, padding='max_length', truncation=True, max_length=256)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs


# Tokenizuj dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)
logger.info("Tokenization completed.")
eval_dataset = tokenized_dataset['validation'] if 'validation' in tokenized_dataset else None
eval_strategy = IntervalStrategy.NO if eval_dataset is None else IntervalStrategy.STEPS

# Proveriti postoji li poslednji validan checkpoint
last_checkpoint = get_last_valid_checkpoint(output_dir)
logger.info(f"Last valid checkpoint: {last_checkpoint}")
print(f"Last valid checkpoint: {last_checkpoint}")

# Oslobodite memoriju pre nego što kreirate instancu trenera
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Definiši parametre za treniranje
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=20,
    per_device_train_batch_size=8,  # Povećani batch size
    per_device_eval_batch_size=4,  # Povećani batch size
    gradient_accumulation_steps=8,  # Smanjeni koraci akumulacije
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=20,
    eval_strategy=eval_strategy,  # Koristimo eval_strategy umesto evaluation_strategy
    save_steps=500,
    save_total_limit=2,
    disable_tqdm=False,
    fp16=True,
    learning_rate=2e-5
)


# Kreiranje podklase `Trainer`
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, *args, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        return (loss, outputs) if return_outputs else loss


# Funkcija za čuvanje istreniranog modela
def save_pretrained_model(model, tokenizer, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    model.save_pretrained(directory)
    tokenizer.save_pretrained(directory)
    with open(os.path.join(directory, 'vocab.txt'), 'w', encoding='utf-8') as vocab_file:
        for token, index in tokenizer.get_vocab().items():
            vocab_file.write(f"{token}\n")
    logger.info(f"Model, tokenizer, and vocabulary saved to {directory}.")


# Kreiraj instancu CustomTrainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=eval_dataset,
)

# Proveriti da li treba nastaviti treniranje od poslednjeg validnog checkpointa
if last_checkpoint:
    logger.info("Resuming training from last valid checkpoint.")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    logger.info("Starting training from scratch.")
    trainer.train()


# Sačuvaj model
logger.info("Saving the trained model...")
save_pretrained_model(model, tokenizer, output_dir)
torch.save(model.state_dict(), os.path.join(output_dir, 'pytorch_model.bin'))

# Proveriti da li je model sačuvan
model_path = os.path.join(output_dir, 'pytorch_model.bin')
if os.path.isfile(model_path):
    logger.info(f"Model saved successfully at {model_path}")
else:
    logger.error(f"Model file not found at {model_path}")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: rudicstipan5 (rudicstipan5-lipilipic). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


exported (1).jsonl:   0%|          | 0.00/3.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/935 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages', 'tools', 'split'],
        num_rows: 935
    })
})


...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/935 [00:00<?, ? examples/s]

Last valid checkpoint: None


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
20,3.527600
40,3.299800
60,2.848000
80,2.132500
100,1.020500
120,0.145600
140,0.006800
160,0.001200
180,0.000700
200,0.000600


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import logging
import torch
import wandb
import os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, IntervalStrategy

# Postavljanje W&B API ključa
os.environ['WANDB_API_KEY'] = '0dd449c9c7a50feb94ef49dbbede869392953b8f'

# Ručni login na W&B ako je potrebno
wandb.login()

# Pokrenite W&B sesiju sa tačnim imenom entiteta i projektom
wandb.init(project="openAI-gpt2", entity="rudicstipan5-lipilipic", settings=wandb.Settings(init_timeout=120))

# Logger setup
logger = logging.getLogger(__name__)
logger.info("Starting script execution...")

# Environment varijabla za izbegavanje fragmentacije memorije CUDA
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Kreiranje potrebnih direktorijuma
output_dir = '/content/drive/My Drive/gpt2-unc'
new_model_output_dir = '/content/drive/My Drive/gpt2-unc-nsfw-unc'
checkpoint_dir = os.path.join(output_dir, new_model_output_dir, 'results')

for directory in [output_dir, checkpoint_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)
        logger.info(f"Created directory: {directory}")

# Funkcija za pronalaženje poslednjeg validnog checkpoint-a
def get_last_valid_checkpoint(directory):
    logger.info(f"Searching for the last valid checkpoint in the output directory {directory}.")
    if not os.path.exists(directory):
        logger.error(f"Output directory {directory} does not exist.")
        return None
    checkpoints = [os.path.join(directory, d) for d in os.listdir(directory) if d.startswith('checkpoint-')]
    if not checkpoints:
        logger.info("No checkpoints found.")
        return None
    last_checkpoint = max(checkpoints, key=os.path.getctime)
    if os.path.isdir(last_checkpoint):
        logger.info(f"Last valid checkpoint: {last_checkpoint}")
        return last_checkpoint
    else:
        logger.info("Valid checkpoint directory not found.")
        return None

# Učitaj dataset
dataset = load_dataset('ResplendentAI/Luna_NSFW_Text')
logger.info("Dataset loaded successfully.")

# Učitaj tokenizer i model sa lokalnog direktorijuma
model_directory = '/content/drive/My Drive/gpt2-UNC-NSFW'
tokenizer = AutoTokenizer.from_pretrained(model_directory)
model = AutoModelForCausalLM.from_pretrained(model_directory)
logger.info("Tokenizer and model loaded from local directory successfully.")
tokenizer.pad_token = tokenizer.eos_token


# Funkcija za tokenizaciju
def tokenize_function(examples):
    inputs = examples['text']  # Pretpostavljamo da je ključ 'text'
    tokenized_inputs = tokenizer(inputs, padding='max_length', truncation=True, max_length=512)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs


# Tokenizuj dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)
logger.info("Tokenization completed.")
eval_dataset = tokenized_dataset['validation'] if 'validation' in tokenized_dataset else None
eval_strategy = IntervalStrategy.NO if eval_dataset is None else IntervalStrategy.STEPS

# Proveriti postoji li poslednji validan checkpoint
last_checkpoint = get_last_valid_checkpoint(output_dir)
logger.info(f"Last valid checkpoint: {last_checkpoint}")
print(f"Last valid checkpoint: {last_checkpoint}")

# Oslobodite memoriju pre nego što kreirate instancu trenera
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Definiši parametre za treniranje
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=20,
    per_device_train_batch_size=4,  # Povećani batch size
    per_device_eval_batch_size=4,  # Povećani batch size
    gradient_accumulation_steps=8,  # Smanjeni koraci akumulacije
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=20,
    eval_strategy=eval_strategy,  # Koristimo eval_strategy
    save_steps=500,
    save_total_limit=2,
    disable_tqdm=False,
    fp16=True,
    learning_rate=2e-5
)


# Kreiranje podklase `Trainer`
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, *args, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        return (loss, outputs) if return_outputs else loss


# Funkcija za čuvanje istreniranog modela
def save_pretrained_model(model, tokenizer, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    model.save_pretrained(directory)
    tokenizer.save_pretrained(directory)
    with open(os.path.join(directory, 'vocab.txt'), 'w', encoding='utf-8') as vocab_file:
        for token, index in tokenizer.get_vocab().items():
            vocab_file.write(f"{token}\n")
    logger.info(f"Model, tokenizer, and vocabulary saved to {directory}.")


# Kreiraj instancu CustomTrainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=eval_dataset,
)

# Proveriti da li treba nastaviti treniranje od poslednjeg validnog checkpointa
if last_checkpoint:
    logger.info("Resuming training from last valid checkpoint.")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    logger.info("Starting training from scratch.")
    trainer.train()


# Sačuvaj model
logger.info("Saving the trained model...")
save_pretrained_model(model, tokenizer, output_dir)
torch.save(model.state_dict(), os.path.join(output_dir, 'pytorch_model.bin'))

# Proveriti da li je model sačuvan
model_path = os.path.join(output_dir, 'pytorch_model.bin')
if os.path.isfile(model_path):
    logger.info(f"Model saved successfully at {model_path}")
else:
    logger.error(f"Model file not found at {model_path}")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: rudicstipan5 (rudicstipan5-lipilipic). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Last valid checkpoint: None


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
20,0.034300
40,0.028900
60,0.031300
80,0.031900
100,0.032600
120,0.033100
140,0.027600
160,0.030300
180,0.033300
200,0.031700


In [ ]:
import logging
import torch
import wandb
import os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, IntervalStrategy

# Postavljanje W&B API ključa
os.environ['WANDB_API_KEY'] = '0dd449c9c7a50feb94ef49dbbede869392953b8f'

# Ručni login na W&B ako je potrebno
wandb.login()

# Pokrenite W&B sesiju sa tačnim imenom entiteta i projektom
wandb.init(project="openAI-gpt2", entity="rudicstipan5-lipilipic", settings=wandb.Settings(init_timeout=120))

# Logger setup
logger = logging.getLogger(__name__)
logger.info("Starting script execution...")

# Environment varijabla za izbegavanje fragmentacije memorije CUDA
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Kreiranje potrebnih direktorijuma
output_dir = '/content/drive/My Drive/gpt2-xxxSR'
new_model_output_dir = '/content/drive/My Drive/gpt2-unc-nsfw-unc'
checkpoint_dir = os.path.join(output_dir, new_model_output_dir, 'results')

for directory in [output_dir, checkpoint_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)
        logger.info(f"Created directory: {directory}")

# Funkcija za pronalaženje poslednjeg validnog checkpoint-a
def get_last_valid_checkpoint(directory):
    logger.info(f"Searching for the last valid checkpoint in the output directory {directory}.")
    if not os.path.exists(directory):
        logger.error(f"Output directory {directory} does not exist.")
        return None
    checkpoints = [os.path.join(directory, d) for d in os.listdir(directory) if d.startswith('checkpoint-')]
    if not checkpoints:
        logger.info("No checkpoints found.")
        return None
    last_checkpoint = max(checkpoints, key=os.path.getctime)
    if os.path.isdir(last_checkpoint):
        logger.info(f"Last valid checkpoint: {last_checkpoint}")
        return last_checkpoint
    else:
        logger.info("Valid checkpoint directory not found.")
        return None

# Učitaj dataset
dataset = load_dataset("saillab/alpaca-croatian-cleaned")
logger.info("Dataset loaded successfully.")
logger.info(f"Dataset columns: {dataset['train'].column_names}")

# Učitaj unapred trenirani model i tokenizer iz direktorijuma
model_directory = '/content/drive/My Drive/gpt2-UNC-NSFW'
tokenizer = AutoTokenizer.from_pretrained(model_directory)
model = AutoModelForCausalLM.from_pretrained(model_directory).to(device)
logger.info("Tokenizer and model loaded from pretrained model successfully.")
tokenizer.pad_token = tokenizer.eos_token


# Funkcija za tokenizaciju
def tokenize_function(examples):
    inputs = [(instruction + " " + input_text).strip() for instruction, input_text in zip(examples['instruction'], examples['input'])]
    tokenized_inputs = tokenizer(inputs, padding='max_length', truncation=True, max_length=512)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs


# Tokenizuj dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)
logger.info("Tokenization completed.")
eval_dataset = tokenized_dataset['validation'] if 'validation' in tokenized_dataset else None
eval_strategy = IntervalStrategy.NO if eval_dataset is None else IntervalStrategy.STEPS

# Proveriti postoji li poslednji validan checkpoint
last_checkpoint = get_last_valid_checkpoint(output_dir)
logger.info(f"Last valid checkpoint: {last_checkpoint}")
print(f"Last valid checkpoint: {last_checkpoint}")

# Oslobodite memoriju pre nego što kreirate instancu trenera
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Definiši parametre za treniranje
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=20,
    per_device_train_batch_size=8,  # Povećani batch size
    per_device_eval_batch_size=4,  # Povećani batch size
    gradient_accumulation_steps=8,  # Smanjeni koraci akumulacije
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=20,
    eval_strategy=eval_strategy,  # Koristimo eval_strategy
    save_steps=500,
    save_total_limit=2,
    disable_tqdm=False,
    fp16=True,
    learning_rate=2e-5
)


# Kreiranje podklase `Trainer`
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, *args, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        return (loss, outputs) if return_outputs else loss


# Funkcija za čuvanje istreniranog modela
def save_pretrained_model(model, tokenizer, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    model.save_pretrained(directory)
    tokenizer.save_pretrained(directory)
    with open(os.path.join(directory, 'vocab.txt'), 'w', encoding='utf-8') as vocab_file:
        for token, index in tokenizer.get_vocab().items():
            vocab_file.write(f"{token}\n")
    logger.info(f"Model, tokenizer, and vocabulary saved to {directory}.")


# Kreiraj instancu CustomTrainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=eval_dataset,
)

# Proveriti da li treba nastaviti treniranje od poslednjeg validnog checkpointa
if last_checkpoint:
    logger.info("Resuming training from last valid checkpoint.")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    logger.info("Starting training from scratch.")
    trainer.train()


# Sačuvaj model
logger.info("Saving the trained model...")
save_pretrained_model(model, tokenizer, output_dir)
torch.save(model.state_dict(), os.path.join(output_dir, 'pytorch_model.bin'))

# Proveriti da li je model sačuvan
model_path = os.path.join(output_dir, 'pytorch_model.bin')
if os.path.isfile(model_path):
    logger.info(f"Model saved successfully at {model_path}")
else:
    logger.error(f"Model file not found at {model_path}")